# Google Trends Exploration

This notebook is to explore pulling recipes from Google Trends and finding ingredients using Spoonacular.

In [9]:
%pip install requests pytrends spoonacular brave-search pandas sentence-transformers scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [10]:
from pytrends.request import TrendReq
from spoonacular import API
import json
import time
import os
from brave import Brave
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/opt/homebrew/Caskroom/miniconda/base/envs/cs4365/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We first setup our API access points.

In [3]:
# Set up Spoonacular API client
spoonacular_api_key = "API_KEY"
spoonacular_client = API(spoonacular_api_key)

# Set up Google Trends API client
pytrends = TrendReq()

# Set up Brave Search API client
brave = Brave('API_KEY')

We then get trending recipes using the Google Trends API and save them.

In [4]:
if not os.path.exists('trending_recipes.json'):
    # Build Google Trends API request with the keyline "recipe" over the past week
    kw_list = ["recipe"]
    pytrends.build_payload(kw_list, cat=0, timeframe="now 7-d", geo="US")
    
    # Extract the rising related queries by week-over-week change of above 50%
    related_queries = pytrends.related_queries()
    trending_recipes = []
    rising_trends = related_queries["recipe"]["rising"]
    for index, row in rising_trends.iterrows():
        keyword = row["query"]
        percentage_change = row["value"]
        trending_recipes.append((keyword, percentage_change))

    trending_recipes_data = {"trending_recipes": trending_recipes}
    with open('trending_recipes.json', "w") as file:
        json.dump(trending_recipes_data, file, indent=4)
else:
    with open('trending_recipes.json') as f:
        data = json.load(f)
    trending_recipes = data["trending_recipes"]

trending_recipes

[['pimento cheese recipe', 300],
 ['pornstar martini recipe', 70],
 ['hummingbird food recipe', 50]]

We see that we get the name of the recipe and we also get the percentage week-over-week growth.

The next step is to get the recipe ingredients. To do this, we query Spoonacular API and see if the recipe is in their database. If it is, we get the ingredients from there. If it isn't, we use Brave Search API to search for a recipe online and using Spoonacular API to scrape the ingredients off the site.

To do this, we need to define a function for the search logic.

In [5]:
def get_recipe_ingredients(recipe_name):
    """
    Extract ingredients from a recipe using the Spoonacular API.
    
    Args:
        recipe_name (str): The name of the recipe.
        
    Returns:
        list: A list of ingredients for the recipe.
    """
    try:
        # Check if the recipe exists in Spoonacular
        search_response = spoonacular_client.search_recipes_complex(recipe_name)
        time.sleep(1)
        if search_response.json()["totalResults"] > 0:
            # If the recipe exists, retrieve its ingredients directly
            recipe_id = search_response.json()["results"][0]["id"]
            ingredients_response = spoonacular_client.get_analyzed_recipe_instructions(recipe_id)
            time.sleep(1)
            steps = ingredients_response.json()[0]["steps"]
            ingredients = []
            for step in steps:
                ingredients.extend(step["ingredients"])
            ingredient_names = [ingredient["name"] for ingredient in ingredients]
            return ingredient_names
        else:
            pass
            # If the recipe doesn't exist, search for a website with the recipe using Brave Search API
            search_query = f"{recipe_name} recipe"
            search_response = brave.search(q=search_query, count=10).web_results
            if len(search_response) > 0:
                recipe_url = search_response[0]["url"]
                
                # Extract ingredients from the recipe URL using Spoonacular
                response = spoonacular_client.extract_recipe_from_website(recipe_url)
                time.sleep(1)
                ingredients = response.json()["extendedIngredients"]
                ingredient_names = [ingredient["name"] for ingredient in ingredients]
                return ingredient_names
    except Exception as e:
        print(f"Error retrieving ingredients for {recipe_name}: {str(e)}")
        return []

This can then be used on each recipe. We do this, and save it to a file.

In [7]:
if not os.path.exists('recipe_ingredients.json'):
    # Get ingredients of each recipe
    recipe_ingredients = {}
    for recipe, _ in trending_recipes:
        ingredients = get_recipe_ingredients(recipe)
        recipe_ingredients[recipe] = ingredients

    recipe_ingredients_data = {"recipe_ingredients": recipe_ingredients}
    with open('trending_recipes.json', "w") as file:
        json.dump(recipe_ingredients_data, 'recipe_ingredients.json', indent=4)
else:
    with open('recipe_ingredients.json') as f:
        data = json.load(f)
    recipe_ingredients = data["recipe_ingredients"]

recipe_ingredients


{'pimento cheese recipe': ['cheddar cheese',
  'cream cheese',
  'pimento peppers**',
  'mayonnaise',
  'garlic powder',
  'onion powder',
  'ground cayenne pepper',
  'jalapeño pepper',
  'pepper',
  'salt'],
 'pornstar martini recipe': ['vanilla vodka',
  'passion fruit liqueur',
  'passion fruit puree',
  'lime juice',
  'vanilla simple syrup',
  'sparkling wine',
  'garnish: 1/ passion fruit'],
 'hummingbird food recipe': ['carrot',
  'baking soda',
  'spices',
  'all purpose flour',
  'salt',
  'olive oil',
  'vanilla',
  'egg',
  'pineapple',
  'banana',
  'carrot',
  'coconut',
  'cream cheese frosting',
  'cream cheese',
  'vanilla',
  'butter',
  'cream',
  'powdered sugar',
  'carrot',
  'vegetable',
  'carrot',
  'granulated sugar',
  'water',
  'carrot',
  'carrot',
  'carrot',
  'carrot',
  'wrap',
  'carrot',
  'cream cheese frosting',
  'frosting',
  'frosting',
  'cream cheese frosting',
  'carrot',
  'pecans']}

The logic works! We are successfully get all the ingredients for every viral/trending recipe.